# Libraries

In [ ]:
#! pip install pyautogen groq
! pip install "autogen-agentchat" "autogen-ext[openai]" rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.2/234.2 kB 17.6 MB/s eta 0:00:00


# Prepare database

In [ ]:
! pwd

/content


In [ ]:
import json
import sqlite3

# Load JSON data from a file
with open("/content/tiki_extracted_products_small.json", "r", encoding="utf-8") as file:
    books = json.load(file)

# Connect to SQLite database (or create if not exists)
conn = sqlite3.connect("books.db")
cursor = conn.cursor()

# Create table if not exists
cursor.execute('''
    CREATE TABLE IF NOT EXISTS books (
        id INTEGER PRIMARY KEY,
        name TEXT,
        type TEXT,
        short_description TEXT,
        price INTEGER,
        original_price INTEGER,
        description TEXT,
        authors TEXT,
        categories TEXT
    )
''')

# Insert data into the table
for book in books:
    authors = book.get("authors", [])  # Get authors, default to empty list if missing
    authors_str = ", ".join(authors) if isinstance(authors, list) else str(authors)

    cursor.execute('''
        INSERT INTO books (id, name, type, short_description, price, original_price, description, authors, categories)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        book["id"],
        book["name"],
        book["type"],
        book["short_description"],
        book["price"],
        book["original_price"],
        book["description"],
        authors_str,
        book["categories"]
    ))

# Commit changes and close connection
conn.commit()
conn.close()

print("Books data saved successfully!")

Books data saved successfully!


In [ ]:
import sqlite3
from datetime import datetime, timedelta
import random

# Connect to the SQLite database
conn = sqlite3.connect("books.db")
cursor = conn.cursor()

# Drop the orders table if it exists
cursor.execute("DROP TABLE IF EXISTS orders")

# Recreate the orders table
cursor.execute("""
    CREATE TABLE orders (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        customer_id INTEGER NOT NULL,
        book_id INTEGER NOT NULL,
        order_date TEXT NOT NULL,
        order_status TEXT CHECK(order_status IN ('Finished', 'In shipping')) NOT NULL
    )
""")

# Sample book IDs provided
book_ids = [416389, 706939, 56002080, 49181801, 54729138,
            275571603, 275571590, 275571109, 275153551, 275062854]

# Generate 10 sample orders for customer_id = 1
customer_id = 1
order_statuses = ["Finished", "In shipping"]
start_date = datetime(2024, 3, 1)  # Example start date

orders = []
for i, book_id in enumerate(book_ids):
    order_date = start_date + timedelta(days=i)  # Incremental order dates
    order_status = random.choice(order_statuses)  # Random status
    orders.append((customer_id, book_id, order_date.strftime("%Y-%m-%d"), order_status))

# Insert orders into the database
cursor.executemany("""
    INSERT INTO orders (customer_id, book_id, order_date, order_status)
    VALUES (?, ?, ?, ?)
""", orders)

# Commit and close connection
conn.commit()
conn.close()

print("Orders table created and populated successfully.")

Orders table created and populated successfully.


In [ ]:
def query_database(query, db_name="books.db"):
    """
    Executes a SQL query on the given SQLite database and returns the results.

    :param db_name: str - Name of the SQLite database file (e.g., 'books.db')
    :param query: str - The SQL query to execute
    :return: list - Query results as a list of tuples
    """
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query with optional parameters
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Close the connection
        conn.close()

        return results

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None

In [ ]:
query = "SELECT DISTINCT categories FROM books"
query_database(query)

[('Business & Economics',),
 ('Personal Improvement',),
 ("Children's Books",),
 ('Dictionary',),
 ('English Language Dictionary',),
 ('Fiction - Literature',),
 ('Psychology',),
 ('Sự lão hóa - trẻ hỏa',),
 ('Root',),
 ('Nhà Sách Tiki',),
 ('Biographies & Memoirs',),
 ('Cẩm nang làm cha mẹ',),
 ('Tiểu Thuyết Phương Tây',),
 ('Luyện thi IELTS',),
 ('Sách tiếng Việt',),
 ('Mystery, Thriller & Suspense',),
 ('Success',),
 ('Books that will change your life',),
 ("Children's Science Fiction & Fantasy",),
 ('Science Fiction & Fantasy',),
 ('Sách Y Học Hiện Đại',),
 ('Châu Á',),
 ('English Books',),
 ('Sách Y Học Khác',),
 ('Phật giáo',),
 ('Sách khoa học',),
 ('Travel & Holiday',),
 ('Thể dục - thể thao - khác',),
 ('Professions',),
 ('Giáo trình đại học - cao đẳng - khác',),
 ('Corporate Finance',),
 ('Đại Trường Phát Educations',),
 ('Kinh tế - Tài chính - Quản trị',),
 ('Giáo trình Hán ngữ',),
 ('Architecture',),
 ('Kỹ Thuật khác',),
 ('Lịch Sử Việt Nam',),
 ('Từ Điển Việt - Anh',),
 ('

# SQL_Agent

In [ ]:
database_schema = {
    "books": {
        "description": "Table storing book details, including metadata and pricing information.",
        "columns": {
            "id": {"type": "INTEGER PRIMARY KEY", "description": "Unique identifier for the book."},
            "name": {"type": "TEXT", "description": "Title of the book."},
            "type": {"type": "TEXT", "description": "Genre or category of the book."},
            "short_description": {"type": "TEXT", "description": "Brief summary of the book."},
            "price": {"type": "REAL", "description": "Current selling price of the book."},
            "original_price": {"type": "REAL", "description": "Original price before discounts."},
            "description": {"type": "TEXT", "description": "Full description of the book."},
            "authors": {"type": "TEXT", "description": "List of authors, stored as a comma-separated string."},
            "categories": {"type": "TEXT", "description": "List of categories, stored as a comma-separated string."}
        }
    },
    "orders": {
        "description": "Table storing customer orders, including book purchases and order status.",
        "columns": {
            "id": {"type": "INTEGER PRIMARY KEY AUTOINCREMENT", "description": "Unique identifier for the order."},
            "customer_id": {"type": "INTEGER", "description": "Unique identifier of the customer placing the order."},
            "book_id": {
                "type": "INTEGER",
                "description": "Unique identifier of the book being ordered. References books(id)."
            },
            "order_date": {"type": "TEXT", "description": "Date when the order was placed (YYYY-MM-DD format)."},
            "order_status": {
                "type": "TEXT CHECK(order_status IN ('Finished', 'In shipping'))",
                "description": "Current status of the order. Can be 'Finished' or 'In shipping'."
            }
        }
    }
}

def generate_schema_prompt(schema):
    prompt = ""
    for table, details in schema.items():
        prompt += f"Table: {table} - {details['description']}\nColumns:\n"
        for column, col_details in details["columns"].items():
            prompt += f"- {column} ({col_details['type']}): {col_details['description']}\n"
        prompt += "\n"
    return prompt

# Generate prompt for the LLM
schema = generate_schema_prompt(database_schema)
print(schema)

Table: books - Table storing book details, including metadata and pricing information.
Columns:
- id (INTEGER PRIMARY KEY): Unique identifier for the book.
- name (TEXT): Title of the book.
- type (TEXT): Genre or category of the book.
- short_description (TEXT): Brief summary of the book.
- price (REAL): Current selling price of the book.
- original_price (REAL): Original price before discounts.
- description (TEXT): Full description of the book.
- authors (TEXT): List of authors, stored as a comma-separated string.
- categories (TEXT): List of categories, stored as a comma-separated string.

Table: orders - Table storing customer orders, including book purchases and order status.
Columns:
- id (INTEGER PRIMARY KEY AUTOINCREMENT): Unique identifier for the order.
- customer_id (INTEGER): Unique identifier of the customer placing the order.
- book_id (INTEGER): Unique identifier of the book being ordered. References books(id).
- order_date (TEXT): Date when the order was placed (YYYY-M

In [ ]:
schema_vie = """
Bảng: books - Bảng lưu trữ thông tin chi tiết về sách, bao gồm siêu dữ liệu và giá bán.
Các cột:
id (INTEGER PRIMARY KEY): Định danh duy nhất của sách.
name (TEXT): Tiêu đề của sách.
short_description (TEXT): Tóm tắt ngắn gọn về sách.
price (REAL): Giá bán hiện tại của sách.
original_price (REAL): Giá gốc trước khi giảm giá.
description (TEXT): Mô tả đầy đủ về sách.
authors (TEXT): Danh sách tác giả, được lưu dưới dạng chuỗi phân tách bằng dấu phẩy.
categories (TEXT): Danh sách danh mục, được lưu dưới dạng chuỗi phân tách bằng dấu phẩy.

Bảng: orders - Bảng lưu trữ đơn hàng của khách hàng, bao gồm thông tin mua sách và trạng thái đơn hàng.
Các cột:
id (INTEGER PRIMARY KEY AUTOINCREMENT): Định danh duy nhất của đơn hàng.
customer_id (INTEGER): Định danh duy nhất của khách hàng đặt hàng.
book_id (INTEGER): Định danh duy nhất của sách được đặt hàng. Tham chiếu đến books(id).
order_date (TEXT): Ngày đặt hàng (định dạng YYYY-MM-DD).
order_status (TEXT CHECK(order_status IN ('Finished', 'In shipping'))): Trạng thái hiện tại của đơn hàng. Có thể là 'Finished' (Hoàn thành) hoặc 'In shipping' (Đang vận chuyển).
"""

In [ ]:
import asyncio
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool

sql_agent_system_message_vie = f"""
Bạn là một trợ lý hữu ích và có thể sử dụng công cụ SQL để truy vấn dữ liệu từ database

Dưới đây là một số điểm quan trọng bạn cần nhớ khi xây dựng truy vấn SQL:
- Khi cần lấy thông tin từ bảng orders, bạn phải lọc dữ liệu chỉ với customer_id = 1.
- Bạn có quyền truy cập vào lược đồ cơ sở dữ liệu sau:
{schema_vie}

Lưu ý:

Khi lọc sách, bạn nên sử dụng toán tử LIKE thay vì toán tử =.
Hãy cố gắng tránh thêm cột `short_description` và `description` vào phần SELECT, chỉ thêm khi thực sự cần thiết.
"""

# tools
def execute_sql_query(query: str):
    import sqlite3
    """
    Executes the given SQL query on the 'books.db' SQLite database and returns the results.

    Args:
        query (str): The SQL query to be executed.

    Returns:
        List[Tuple]: A list of tuples containing the rows returned by the query.
    """
    # Connect to the SQLite database
    conn = sqlite3.connect('/content/books.db')
    cursor = conn.cursor()

    try:
        # Execute the SQL query
        cursor.execute(query)
        # Fetch all results
        results = cursor.fetchall()
        return results
    except sqlite3.Error as e:
        # Handle SQL errors
        print(f"An error occurred: {e}")
        return []
    finally:
        # Close the database connection
        conn.close()

database_query_tool = FunctionTool(
    execute_sql_query, description="Truy vấn thông tin về sách từ database sử dụng SQL."
)


model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key='',
    seed=42,
    temperature=0)

sql_database_assistant = AssistantAgent(
    name="sql_database_assistant",
    description="Agent hỗ trợ trả lời các câu hỏi liên quan đến thông tin sách và thông tin mua hàng của khách hàng.",
    system_message=sql_agent_system_message_vie,
    model_client=model_client,
    tools=[database_query_tool],
    reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
)
user_input = "Giá của đơn hàng cuối cùng của tôi là bao nhiêu?"
response = await sql_database_assistant.on_messages([TextMessage(content=user_input, source="user")], CancellationToken())
print("Assistant:", response.chat_message.content)
print(response.inner_messages)

Assistant: Giá của đơn hàng cuối cùng của bạn là 81,880.
[ToolCallRequestEvent(source='sql_database_assistant', models_usage=RequestUsage(prompt_tokens=518, completion_tokens=48), metadata={}, content=[FunctionCall(id='call_kcb4Hs0Z0GdS1pZkUIMYDTa8', arguments='{"query":"SELECT b.price FROM orders o JOIN books b ON o.book_id = b.id WHERE o.customer_id = 1 ORDER BY o.order_date DESC LIMIT 1;"}', name='execute_sql_query')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_database_assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='[(81880,)]', name='execute_sql_query', call_id='call_kcb4Hs0Z0GdS1pZkUIMYDTa8', is_error=False)], type='ToolCallExecutionEvent')]


# policy agent

In [ ]:
! pip install langchain-community langchain -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
! pip install chromadb sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.3 MB/s eta 0:0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder
from langchain.schema import Document
import os
import numpy as np
import unicodedata
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
from datetime import datetime

In [ ]:
# Load a single text file
file_path = r'drive/MyDrive/nlp/Policy/tikinow.txt'
loader = TextLoader(file_path, encoding='utf-8')
documents = loader.load()

for doc in documents:
    doc.page_content = unicodedata.normalize("NFKC", doc.page_content)

In [ ]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "],  # Tries splitting by paragraph, sentence, then word
    keep_separator=False
)

chunks = text_splitter.split_documents(documents)

In [ ]:
for chunk in chunks:
    print(chunk.page_content)
    print('---------')

﻿Dịch vụ giao hàng TikiNOW


Khi cần Tiki có, khi khó Tiki giao! Giờ đây quý khách đã có thêm lựa chọn giao hàng khi mua sắm tại Tiki – Giao nhanh tại TP. Hồ Chí Minh/ Hà Nội với hàng trăm ngàn sản phẩm TikiNOW.


1. Đối tượng áp dụng:
Các khách hàng có đăng nhập để đặt hàng và trong khu vực có hỗ trợ dịch vụ.
---------
1. Đối tượng áp dụng:
Các khách hàng có đăng nhập để đặt hàng và trong khu vực có hỗ trợ dịch vụ.


2. Nội dung chi tiết:
 
 NỘI DUNG         THÔNG TIN
 Dấu hiệu nhận biết        NOW logo-primary.png
 Phí vận chuyển         Giao Siêu Tốc 2H: 25.000 VND/3kg (+2.000 VND/ mỗi 0.5kg tiếp theo)
 BookCare         Chỉ hỗ trợ đơn hàng dưới 20 quyển sách
---------
3. Khung giờ giao hàng:
Quý khách vui lòng kiểm tra thời gian giao hàng dự kiến tại mục thông tin vận chuyển ở Bước "Thanh toán".


- Nếu đơn hàng được đặt trước 18h: Sẽ được giao trong 2h-3h kể từ lúc đặt hàng.


- Nếu đơn hàng được đặt sau 18h: Sẽ được giao trong buổi sáng tiếp theo.


4. Khu vực áp dụng dịch vụ:
Quý

In [ ]:
%cd /content/drive/MyDrive/nlp

/content/drive/MyDrive/nlp


In [ ]:
import os
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize ChromaDB client & collection
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_client.delete_collection("tiki_doc")

collection = chroma_client.get_or_create_collection("tiki_doc")

# Load embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

def chunk_and_store_document(file_path):
    """Reads a document, splits into chunks, embeds, and stores them in ChromaDB."""
    doc_name = os.path.splitext(os.path.basename(file_path))[0]  # Extract document name

    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()

    # Text chunking
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400, chunk_overlap=100,
        separators=["\n\n", "\n", ".", " "], keep_separator=False
    )
    chunks = text_splitter.create_documents([text])  # Split into chunks

    # Embed and store each chunk separately
    for idx, chunk in enumerate(chunks):
        chunk_text = chunk.page_content
        embedding = model.encode(chunk_text).tolist()

        chunk_id = f"{doc_name}_{idx}"  # Unique ID for each chunk
        metadata = {
            "document_name": doc_name,
            "filename": file_path,
            "chunk_id": idx,
            "created_at": datetime.utcnow().isoformat(),  # Store timestamp
        }

        collection.add(
            ids=[chunk_id],
            embeddings=[embedding],
            documents=[chunk_text],
            metadatas=[metadata]  # Storing metadata only
        )

    print(f"✅ Stored {len(chunks)} chunks from '{file_path}' in ChromaDB!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import os

# Define the directory containing the text files
directory = "./Policy"

# Iterate through all .txt files in the directory
for file_name in os.listdir(directory):
    if file_name.endswith(".txt"):  # Process only .txt files
        file_path = os.path.join(directory, file_name)  # Get full file path
        doc_name = os.path.splitext(file_name)[0]  # Extract document name without extension

        print(f"Processing: {file_name} ...")  # Optional: Print progress
        chunk_and_store_document(file_path)  # Process the file

print("All text files have been processed and stored.")

Processing: theo_doi_doi_tra.txt ...
✅ Stored 3 chunks from './Policy/theo_doi_doi_tra.txt' in ChromaDB!
Processing: kiem_hang.txt ...
✅ Stored 3 chunks from './Policy/kiem_hang.txt' in ChromaDB!
Processing: kich_hoat_bao_hanh.txt ...
✅ Stored 5 chunks from './Policy/kich_hoat_bao_hanh.txt' in ChromaDB!
Processing: huong_dan_dong_goi_hang_tra.txt ...
✅ Stored 9 chunks from './Policy/huong_dan_dong_goi_hang_tra.txt' in ChromaDB!
Processing: noi_tiep_nhan_doi_tra.txt ...
✅ Stored 6 chunks from './Policy/noi_tiep_nhan_doi_tra.txt' in ChromaDB!
Processing: huong_dan_doi_tra_online.txt ...
✅ Stored 6 chunks from './Policy/huong_dan_doi_tra_online.txt' in ChromaDB!
Processing: huong_dan_dat_voucher.txt ...
✅ Stored 3 chunks from './Policy/huong_dan_dat_voucher.txt' in ChromaDB!
Processing: qa_doi_tra.txt ...
✅ Stored 16 chunks from './Policy/qa_doi_tra.txt' in ChromaDB!
Processing: san_pham_khong_doi_tra.txt ...
✅ Stored 5 chunks from './Policy/san_pham_khong_doi_tra.txt' in ChromaDB!
Proces

In [ ]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection("tiki_doc")

In [ ]:
chroma_client.list_collections()

['tiki_doc']

In [ ]:
def list_all_chunks():
    """Lists all stored chunks in ChromaDB with their full details."""

    results = collection.get(include=["documents", "embeddings", "metadatas"])  # Fetch everything

    if not results.get("ids"):
        print("No chunks found in ChromaDB.")
        return

    print(f"Found {len(results['ids'])} chunks in ChromaDB:\n")
    print("-" * 50)

    for i, chunk_id in enumerate(results["ids"]):
        metadata = results["metadatas"][i] if results.get("metadatas") and results["metadatas"] else {}

        print(f"Chunk ID: {chunk_id}")
        print(f"Metadata: {metadata}")
        print("-" * 50)

# Example Usage
list_all_chunks()  # Lists all stored chunks in the collection

Found 205 chunks in ChromaDB:

--------------------------------------------------
Chunk ID: theo_doi_doi_tra_0
Metadata: {'chunk_id': 0, 'created_at': '2025-03-21T16:18:54.526016', 'document_name': 'theo_doi_doi_tra', 'filename': './Policy/theo_doi_doi_tra.txt'}
--------------------------------------------------
Chunk ID: theo_doi_doi_tra_1
Metadata: {'chunk_id': 1, 'created_at': '2025-03-21T16:18:54.894527', 'document_name': 'theo_doi_doi_tra', 'filename': './Policy/theo_doi_doi_tra.txt'}
--------------------------------------------------
Chunk ID: theo_doi_doi_tra_2
Metadata: {'chunk_id': 2, 'created_at': '2025-03-21T16:18:55.075508', 'document_name': 'theo_doi_doi_tra', 'filename': './Policy/theo_doi_doi_tra.txt'}
--------------------------------------------------
Chunk ID: kiem_hang_0
Metadata: {'chunk_id': 0, 'created_at': '2025-03-21T16:18:55.341878', 'document_name': 'kiem_hang', 'filename': './Policy/kiem_hang.txt'}
--------------------------------------------------
Chunk ID: k

In [ ]:
def delete_chunks_by_prefix(prefix):
    """Deletes all chunks from ChromaDB where the ID starts with the given prefix."""

    # Retrieve all stored metadata (including IDs)
    results = collection.get(include=["metadatas"])

    # Extract IDs that match the prefix
    chunk_ids_to_delete = [
        chunk_id for chunk_id in results["ids"] if chunk_id.startswith(prefix)
    ]

    # Delete matching chunks
    if chunk_ids_to_delete:
        collection.delete(ids=chunk_ids_to_delete)
        print(f"Deleted {len(chunk_ids_to_delete)} chunks with prefix '{prefix}' from ChromaDB.")
    else:
        print(f"No chunks found with prefix '{prefix}'.")

# # Example Usage
# delete_chunks_by_prefix("chinh_sach_giao_hang")  # Deletes all chunks where ID starts with 'tikifast'

In [ ]:
# Load embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

def retrieve_chunks(query, top_k=5):
    """Retrieves the top_k most relevant chunks from ChromaDB based on the query."""

    # Generate embedding for the query
    query_embedding = model.encode(query).tolist()

    # Search for relevant chunks in ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=["metadatas", "documents"]
    )

    # Check if any documents were retrieved
    if not results.get("documents") or not results["documents"][0]:
        print("No relevant chunks found.")
        return []

    retrieved_chunks = []

    # Iterate through retrieved chunks
    for doc, metadata in zip(results["documents"][0], results.get("metadatas", [[]])[0]):
        metadata = metadata or {}  # Ensure metadata is a dictionary
        retrieved_chunks.append({"text": doc, "metadata": metadata})

    return retrieved_chunks

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
! pip install openai

In [ ]:
import openai

# Initialize OpenAI client
client = openai.OpenAI(api_key="")

def rewrite_query(query):
    """Rewrites a query for better retrieval in RAG."""

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # OpenAI's GPT-4o-mini model
        messages=[{"role": "user", "content": f"Hãy viết lại câu query sau bằng tiếng Việt để hỗ trợ RAG chatbot: {query}"}],
        temperature=1,
        max_tokens=200
    )
    return response.choices[0].message.content

# Example usage
query = "How does AI impact business decision-making?"
print(rewrite_query(query))

Cách mà trí tuệ nhân tạo ảnh hưởng đến việc ra quyết định trong kinh doanh là gì?


In [ ]:
def iterative_retrieval(query, iter=3):
    """Rewrite query iteratively, retrieve results, and remove duplicates."""

    seen_texts = set()
    retrieved_chunks = []

    for _ in range(iter):
        query = rewrite_query(query)
        chunks = retrieve_chunks(query)

        for chunk in chunks:
            text = chunk["text"]
            metadata = chunk["metadata"]

            # Only add chunks we haven't seen before
            if text not in seen_texts:
                seen_texts.add(text)
                retrieved_chunks.append({"text": text, "metadata": metadata})

    return retrieved_chunks

In [ ]:
def reranker(query, chunks):
    """Reranks retrieved chunks using a cross-encoder."""

    cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    if not chunks:
        return []

    pairs = [(query, chunk["text"]) for chunk in chunks]
    scores = cross_encoder.predict(pairs)

    # Sort by score in descending order
    reranked_chunks = sorted(zip(chunks, scores), key=lambda x: x[1], reverse=True)

    # Return reranked chunks with scores
    return [{"text": chunk["text"], "metadata": chunk["metadata"], "score": score} for chunk, score in reranked_chunks]

In [ ]:
def fetch_policies(query: str):
    retrieved_chunks = iterative_retrieval(query)
    reranked_chunks = reranker(query, retrieved_chunks)
    return_string = ""
    for chunk in reranked_chunks:
        return_string += chunk["text"] + "\n"
    return return_string

In [ ]:
fetch_policies("policy trả hàng")

'\ufeffChính sách hậu mãi: Đổi mới, trả hàng hoàn tiền và bảo hành sản phẩm\n1. PHẠM VI VÀ ĐỐI TƯỢNG ÁP DỤNG\na. Đối tượng áp dụng\nChính Sách Hậu Mãi bao gồm đổi mới, trả hàng hoàn tiền và bảo hành sản phẩm (“Chính Sách Hậu Mãi”)  tại Tiki sẽ áp dụng đối với Khách Hàng khi thực hiện yêu cầu đổi mới, trả hàng, hoàn tiền và bảo hành sản phẩm mà Khách Hàng đã mua trên Sàn TMĐT Ti Ki.\nb. Phạm vi áp dụng\nb. Phạm vi áp dụng\nChính Sách Hậu Mãi tại Tiki sẽ quy định về quyền và nghĩa vụ của Khách Hàng trong việc đổi mới, trả hàng, hoàn tiền và bảo hành sản phẩm mà Khách Hàng đã mua trên Sàn TMĐT Ti Ki.\n3. Chính sách\na. Chính sách đặt hàng:\n \n* Giá bán sản phẩm đã bao gồm các phí, thuế phát sinh trong quá trình vận chuyển thông quan về Việt Nam và chưa bao gồm phí vận chuyển chặng cuối giao hàng trong lãnh thổ Việt Nam. Phí vận chuyển giao hàng trong lãnh thổ Việt Nam sẽ áp dụng theo chính sách giao hàng chung của Tiki đối với khách hàng đặt hàng.\n* Khách hàng có thể nhận diện sản phẩm 

In [ ]:
policy_agent_system_message_vie = """
Bạn là một trợ lý hữu ích và có thể sử dụng công cụ để lấy thông tin về các chính sách của hiệu sách.
Nhiệm vụ của bạn là trả lời các câu hỏi liên quan đến chính sách.
"""
fetch_policies_tool = FunctionTool(
    fetch_policies, description="Lấy thông tin về các chính sách của hiệu sách."
)

policy_agent = AssistantAgent(
    name="policy_agent",
    description="Agent hỗ trợ trả lời các câu hỏi liên quan đến chính sách.",
    system_message=policy_agent_system_message_vie,
    model_client=model_client,
    tools=[fetch_policies_tool],
    reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
)

user_input = "Tôi đã đọc cuốn sách này xong rồi tuy nhiên không thích và muốn hoàn lại."
response = await policy_agent.on_messages([TextMessage(content=user_input, source="user")], CancellationToken())
print("Assistant:", response.chat_message.content)
print(response.inner_messages)

Assistant: Để hoàn trả cuốn sách mà bạn không thích, bạn cần đảm bảo rằng sản phẩm còn nguyên tình trạng như khi nhận, bao gồm hộp và bao bì đầy đủ, chưa qua sử dụng, và còn nguyên tem, mác hoặc niêm phong của nhà sản xuất (nếu có). Ngoài ra, bạn cũng cần kiểm tra xem cuốn sách đó có nằm trong danh mục hạn chế đổi trả hay không.

Nếu bạn đáp ứng đủ các điều kiện trên, bạn có thể thực hiện yêu cầu hoàn trả theo chính sách của hiệu sách. Hãy chắc chắn rằng bạn thực hiện yêu cầu trong thời hạn quy định. Nếu cần thêm thông tin chi tiết, bạn có thể tham khảo trang chính sách đổi trả của hiệu sách.
[ToolCallRequestEvent(source='policy_agent', models_usage=RequestUsage(prompt_tokens=122, completion_tokens=19), metadata={}, content=[FunctionCall(id='call_8pDZsxmQRrPvP34OLMsu0E8d', arguments='{"query":"hoàn trả sách"}', name='fetch_policies')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='policy_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='\u

# GroupChat

In [ ]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

selector_prompt = """Chọn một agent để thực hiện nhiệm vụ.

{roles}

Ngữ cảnh cuộc trò chuyện hiện tại:
{history}

Đọc cuộc trò chuyện ở trên, sau đó chọn một agent từ {participants} để thực hiện nhiệm vụ tiếp theo.
Chỉ chọn một tác nhân duy nhất.
"""

In [ ]:
summarize_agent = AssistantAgent(
    "SummarizeAgent",
    model_client=model_client,
    system_message="""
    Bạn là đại diện của cửa hàng, người mà sẽ đưa ra câu trả lời cuối cùng cho khách hàng.
    Bạn sẽ nhận được thông tin từ các assistant agents.
    Nếu thông tin bạn nhận được liên quan đến quyển sách nào đó, bạn PHẢI yêu cầu một cách lịch sự khách hàng mua quyển sách đó.
    Còn nếu không, hãy tổng hợp lại thông tin từ các assistant agents và đưa ra câu trả lời một cách lịch sự cho khách hàng.
    Hãy trả lời "TERMINATE" khi kết thúc.
    """,
)

def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    # print(messages[-1].source)
    if messages[-1].source in ("sql_database_assistant", "policy_agent"):
        return summarize_agent.name
    if messages[-1].source == "user":
        return None

# Reset the previous team and run the chat again with the selector function.
# await team.reset()
team = SelectorGroupChat(
    [sql_database_assistant, policy_agent, summarize_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

task = "Tài xế cọc cằn, đánh khách hàng khi giao sách. Tôi muốn khiếu nại!"
await Console(team.run_stream(task=task))

---------- user ----------
Tài xế cọc cằn, đánh khách hàng khi giao sách. Tôi muốn khiếu nại!
---------- policy_agent ----------
[FunctionCall(id='call_KW5mFQaQYJgqz241Oitvx1xL', arguments='{"query":"khiếu nại tài xế"}', name='fetch_policies')]
---------- policy_agent ----------
[FunctionExecutionResult(content='* Người mua gửi khiếu nại tại địa chỉ:\nCông ty TNHH Ti Ki\nĐịa chỉ: 52 Út Tịch, phường 4, quận Tân Bình, thành phố Hồ Chí Minh\nQuý khách vui lòng đối chiếu thông tin mã thu hồi với nhân viên vận chuyển trước khi đưa hàng để đảm bảo đúng kiện hàng.\nĐể bảo vệ quyền lợi và tránh thất lạc hàng hóa, quý khách chỉ bàn giao kiện hàng cho nhân viên vận chuyển khi đảm bảo có 1 trong các điều kiện sau:\n- Nhân viên vận chuyển cung cấp giấy xác nhận thu hồi.\n2. ĐIỀU KIỆN ÁP DỤNG\nYêu cầu đổi trả, bảo hành của Khách Hàng được xem là hợp lệ khi đơn hàng và sản phẩm thỏa mãn các tiêu chí:\n(i) Khách Hàng yêu cầu thực hiện chính sách Hậu Mãi trong thời hạn quy định và điều kiện tại https:

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Tài xế cọc cằn, đánh khách hàng khi giao sách. Tôi muốn khiếu nại!', type='TextMessage'), ToolCallRequestEvent(source='policy_agent', models_usage=RequestUsage(prompt_tokens=1213, completion_tokens=23), metadata={}, content=[FunctionCall(id='call_KW5mFQaQYJgqz241Oitvx1xL', arguments='{"query":"khiếu nại tài xế"}', name='fetch_policies')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='policy_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='* Người mua gửi khiếu nại tại địa chỉ:\nCông ty TNHH Ti Ki\nĐịa chỉ: 52 Út Tịch, phường 4, quận Tân Bình, thành phố Hồ Chí Minh\nQuý khách vui lòng đối chiếu thông tin mã thu hồi với nhân viên vận chuyển trước khi đưa hàng để đảm bảo đúng kiện hàng.\nĐể bảo vệ quyền lợi và tránh thất lạc hàng hóa, quý khách chỉ bàn giao kiện hàng cho nhân viên vận chuyển khi đảm bảo có 1 trong các điều kiện sau:\n- Nhân viên vận chuy

In [ ]:
task = "Tôi hủy đơn của cửa hàng bạn liên tiếp 10 lần rồi, liệu tôi có bị vào danh sách đen không?"
await Console(team.run_stream(task=task))

---------- user ----------
Tôi hủy đơn của cửa hàng bạn liên tiếp 10 lần rồi, liệu tôi có bị vào danh sách đen không?
---------- policy_agent ----------
[FunctionCall(id='call_SuSHTYVj8WePj00dLSt98ppu', arguments='{"query":"hủy đơn hàng"}', name='fetch_policies')]
---------- policy_agent ----------
[FunctionExecutionResult(content='c. Chính sách hủy và hoàn tiền:\n   * Đối với đơn hàng có giá trị dưới 1.000.000 VNĐ: Tiki sẽ hoàn tiền toàn bộ giá trị đơn hàng với mọi lý do hủy đơn hàng.\n   * Đối với đơn hàng có giá trị trên 1.000.000 VNĐ:\n     + Nếu đơn hàng chưa thông quan về Việt Nam: Tiki hỗ trợ hủy đơn, hoàn toàn bộ giá trị đơn hàng cho khách hàng với bất kỳ lý do gì.\n+ Nếu đơn hàng đã thông quan về Việt Nam: Tiki vẫn hỗ trợ hủy và hoàn lại giá trị đã thanh toán thực tế cho sản phẩm và phí vận chuyển (nếu có). Tuy nhiên, phần thuế phí nhập khẩu sẽ không được hoàn lại (trừ trường hợp yêu cầu hủy đơn hàng đến từ phía Tiki).\n   Ngày hiệu lực chính sách 24/05/2018, cập nhật lần cuối

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Tôi hủy đơn của cửa hàng bạn liên tiếp 10 lần rồi, liệu tôi có bị vào danh sách đen không?', type='TextMessage'), ToolCallRequestEvent(source='policy_agent', models_usage=RequestUsage(prompt_tokens=2199, completion_tokens=19), metadata={}, content=[FunctionCall(id='call_SuSHTYVj8WePj00dLSt98ppu', arguments='{"query":"hủy đơn hàng"}', name='fetch_policies')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='policy_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='c. Chính sách hủy và hoàn tiền:\n   * Đối với đơn hàng có giá trị dưới 1.000.000 VNĐ: Tiki sẽ hoàn tiền toàn bộ giá trị đơn hàng với mọi lý do hủy đơn hàng.\n   * Đối với đơn hàng có giá trị trên 1.000.000 VNĐ:\n     + Nếu đơn hàng chưa thông quan về Việt Nam: Tiki hỗ trợ hủy đơn, hoàn toàn bộ giá trị đơn hàng cho khách hàng với bất kỳ lý do gì.\n+ Nếu đơn hàng đã thông quan về Việt Nam: Tiki vẫn 

In [ ]:
task = "Hôm nay ăn gì?"
await Console(team.run_stream(task=task))

---------- user ----------
Hôm nay ăn gì?
---------- SummarizeAgent ----------
Xin lỗi, nhưng tôi không thể cung cấp thông tin về thực phẩm hoặc gợi ý món ăn. Tuy nhiên, nếu bạn cần thông tin về một quyển sách nào đó, tôi rất sẵn lòng giúp đỡ! Bạn có muốn tìm hiểu về một quyển sách nào không? 

TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Hôm nay ăn gì?', type='TextMessage'), TextMessage(source='SummarizeAgent', models_usage=RequestUsage(prompt_tokens=980, completion_tokens=64), metadata={}, content='Xin lỗi, nhưng tôi không thể cung cấp thông tin về thực phẩm hoặc gợi ý món ăn. Tuy nhiên, nếu bạn cần thông tin về một quyển sách nào đó, tôi rất sẵn lòng giúp đỡ! Bạn có muốn tìm hiểu về một quyển sách nào không? \n\nTERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")

In [ ]:
task = "Hãng xe nào tốt nhất Việt Nam?"
await Console(team.run_stream(task=task))

---------- user ----------
Hãng xe nào tốt nhất Việt Nam?
---------- SummarizeAgent ----------
Cảm ơn bạn đã đặt câu hỏi! Tuy nhiên, câu hỏi của bạn không liên quan đến sản phẩm hoặc dịch vụ của cửa hàng chúng tôi. Nếu bạn cần tư vấn về sách hoặc sản phẩm nào khác, hãy cho tôi biết, tôi rất sẵn lòng giúp đỡ!

TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Hãng xe nào tốt nhất Việt Nam?', type='TextMessage'), TextMessage(source='SummarizeAgent', models_usage=RequestUsage(prompt_tokens=913, completion_tokens=63), metadata={}, content='Cảm ơn bạn đã đặt câu hỏi! Tuy nhiên, câu hỏi của bạn không liên quan đến sản phẩm hoặc dịch vụ của cửa hàng chúng tôi. Nếu bạn cần tư vấn về sách hoặc sản phẩm nào khác, hãy cho tôi biết, tôi rất sẵn lòng giúp đỡ!\n\nTERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")

In [ ]:
task = "Tên sách của đơn hàng gần nhất của tôi là gì?"
await Console(team.run_stream(task=task))

---------- user ----------
Tên sách của đơn hàng gần nhất của tôi là gì?
---------- sql_database_assistant ----------
[FunctionCall(id='call_iNdzceqGbLVLspBFVrgQd11Q', arguments='{"query":"SELECT b.name FROM orders o JOIN books b ON o.book_id = b.id WHERE o.customer_id = 1 ORDER BY o.order_date DESC LIMIT 1;"}', name='execute_sql_query')]
---------- sql_database_assistant ----------
[FunctionExecutionResult(content="[('Bài tập Bổ trợ Tiếng Anh 9 Right On!',)]", name='execute_sql_query', call_id='call_iNdzceqGbLVLspBFVrgQd11Q', is_error=False)]
---------- sql_database_assistant ----------
Tên sách của đơn hàng gần nhất của bạn là "Bài tập Bổ trợ Tiếng Anh 9 Right On!".
---------- SummarizeAgent ----------
Tên sách của đơn hàng gần nhất của bạn là "Bài tập Bổ trợ Tiếng Anh 9 Right On!". Nếu bạn quan tâm, bạn có thể mua quyển sách này hoặc tìm hiểu thêm về nó. Nếu bạn cần thêm thông tin, hãy cho tôi biết nhé!

TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Tên sách của đơn hàng gần nhất của tôi là gì?', type='TextMessage'), ToolCallRequestEvent(source='sql_database_assistant', models_usage=RequestUsage(prompt_tokens=1487, completion_tokens=48), metadata={}, content=[FunctionCall(id='call_iNdzceqGbLVLspBFVrgQd11Q', arguments='{"query":"SELECT b.name FROM orders o JOIN books b ON o.book_id = b.id WHERE o.customer_id = 1 ORDER BY o.order_date DESC LIMIT 1;"}', name='execute_sql_query')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_database_assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="[('Bài tập Bổ trợ Tiếng Anh 9 Right On!',)]", name='execute_sql_query', call_id='call_iNdzceqGbLVLspBFVrgQd11Q', is_error=False)], type='ToolCallExecutionEvent'), TextMessage(source='sql_database_assistant', models_usage=RequestUsage(prompt_tokens=1519, completion_tokens=26), metadata={}, content='Tên sách của đơ

In [ ]:
task = "hãy giới thiệu cho tôi 5 quyển sách thuộc danh mục luật"
await Console(team.run_stream(task=task))

---------- user ----------
hãy giới thiệu cho tôi 5 quyển sách thuộc danh mục luật
---------- sql_database_assistant ----------
Dưới đây là 5 quyển sách thuộc danh mục luật mà bạn có thể tham khảo:

1. Hệ Thống Toàn Văn Luật Đất Đai Và Các Nghị Định, Thông Tư, Quyết Định Hướng Dẫn Thi Hành
2. Pháp luật về môi giới, đầu tư kinh doanh bất động sản - nhà ở và đất đai
3. 101 Tư Vấn Pháp Luật Thường Thức Về Đất Đai (Tái Bản Lần 3 - 2020)
4. Pháp Luật Về Hợp Đồng - Các Vấn Đề Pháp Lý Cơ Bản - LS Trương Nhật Quang
5. Bộ Luật Lao Động Hiện Hành (Mới Nhất) Thông Qua Tại Kỳ Họp Thứ 8 Quốc Hội Khóa XIV

Nếu bạn cần thêm thông tin về bất kỳ quyển sách nào, hãy cho tôi biết!
---------- SummarizeAgent ----------
Dưới đây là 5 quyển sách thuộc danh mục luật mà bạn có thể tham khảo:

1. **Hệ Thống Toàn Văn Luật Đất Đai Và Các Nghị Định, Thông Tư, Quyết Định Hướng Dẫn Thi Hành**
2. **Pháp luật về môi giới, đầu tư kinh doanh bất động sản - nhà ở và đất đai**
3. **101 Tư Vấn Pháp Luật Thường Thức Về Đất 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='hãy giới thiệu cho tôi 5 quyển sách thuộc danh mục luật', type='TextMessage'), TextMessage(source='sql_database_assistant', models_usage=RequestUsage(prompt_tokens=4147, completion_tokens=212), metadata={}, content='Dưới đây là 5 quyển sách thuộc danh mục luật mà bạn có thể tham khảo:\n\n1. Hệ Thống Toàn Văn Luật Đất Đai Và Các Nghị Định, Thông Tư, Quyết Định Hướng Dẫn Thi Hành\n2. Pháp luật về môi giới, đầu tư kinh doanh bất động sản - nhà ở và đất đai\n3. 101 Tư Vấn Pháp Luật Thường Thức Về Đất Đai (Tái Bản Lần 3 - 2020)\n4. Pháp Luật Về Hợp Đồng - Các Vấn Đề Pháp Lý Cơ Bản - LS Trương Nhật Quang\n5. Bộ Luật Lao Động Hiện Hành (Mới Nhất) Thông Qua Tại Kỳ Họp Thứ 8 Quốc Hội Khóa XIV\n\nNếu bạn cần thêm thông tin về bất kỳ quyển sách nào, hãy cho tôi biết!', type='TextMessage'), TextMessage(source='SummarizeAgent', models_usage=RequestUsage(prompt_tokens=1291, completion_tokens=245), metadata={}, c

In [ ]:
task = "tổng giá trị 3 đơn hàng gần nhất của tôi là bao nhiêu?"
await Console(team.run_stream(task=task))

---------- user ----------
tổng giá trị 3 đơn hàng gần nhất của tôi là bao nhiêu?
---------- sql_database_assistant ----------
[FunctionCall(id='call_7dy4mdj6xuRie2Trbz9JBprI', arguments='{"query":"SELECT b.price FROM orders o JOIN books b ON o.book_id = b.id WHERE o.customer_id = 1 ORDER BY o.order_date DESC LIMIT 3;"}', name='execute_sql_query')]
---------- sql_database_assistant ----------
[FunctionExecutionResult(content='[(81880,), (208000,), (139000,)]', name='execute_sql_query', call_id='call_7dy4mdj6xuRie2Trbz9JBprI', is_error=False)]
---------- sql_database_assistant ----------
Tổng giá trị của 3 đơn hàng gần nhất của bạn là 81,880 + 208,000 + 139,000 = 428,880.
---------- SummarizeAgent ----------
Tổng giá trị của 3 đơn hàng gần nhất của bạn là 428,880. Nếu bạn cần thêm thông tin hoặc hỗ trợ gì khác, hãy cho tôi biết nhé!

TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='tổng giá trị 3 đơn hàng gần nhất của tôi là bao nhiêu?', type='TextMessage'), ToolCallRequestEvent(source='sql_database_assistant', models_usage=RequestUsage(prompt_tokens=2701, completion_tokens=48), metadata={}, content=[FunctionCall(id='call_7dy4mdj6xuRie2Trbz9JBprI', arguments='{"query":"SELECT b.price FROM orders o JOIN books b ON o.book_id = b.id WHERE o.customer_id = 1 ORDER BY o.order_date DESC LIMIT 3;"}', name='execute_sql_query')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_database_assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='[(81880,), (208000,), (139000,)]', name='execute_sql_query', call_id='call_7dy4mdj6xuRie2Trbz9JBprI', is_error=False)], type='ToolCallExecutionEvent'), TextMessage(source='sql_database_assistant', models_usage=RequestUsage(prompt_tokens=2729, completion_tokens=35), metadata={}, content='Tổng giá trị của

In [ ]:
task = "hãy giới thiệu cho tôi 5 quyển sách thuộc danh mục phật giáo"
await Console(team.run_stream(task=task))

---------- user ----------
hãy giới thiệu cho tôi 5 quyển sách thuộc danh mục phật giáo
---------- sql_database_assistant ----------
[FunctionCall(id='call_IMc3lVesarSNp81a93YTwBMM', arguments='{"query":"SELECT name FROM books WHERE categories LIKE \'%phật giáo%\' LIMIT 5;"}', name='execute_sql_query')]
---------- sql_database_assistant ----------
[FunctionExecutionResult(content="[('Phật Thuyết Đại Thừa Vô Lượng Thọ Trang Nghiêm Thanh Tịnh Bình Đẳng Giác Kinh (Quyển 1)',), ('Lược Sử Tôn Giáo',), ('Sách Muôn Kiếp Nhân Sinh (Bìa Mềm) - Nguyên Phong',), ('Thiền Sư Và Em Bé 5 Tuổi',), ('Hành Trình Về Phương Đông (Tái Bản 2021)',)]", name='execute_sql_query', call_id='call_IMc3lVesarSNp81a93YTwBMM', is_error=False)]
---------- sql_database_assistant ----------
Dưới đây là 5 quyển sách thuộc danh mục Phật giáo mà bạn có thể tham khảo:

1. Phật Thuyết Đại Thừa Vô Lượng Thọ Trang Nghiêm Thanh Tịnh Bình Đẳng Giác Kinh (Quyển 1)
2. Lược Sử Tôn Giáo
3. Sách Muôn Kiếp Nhân Sinh (Bìa Mềm) - Nguyên

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='hãy giới thiệu cho tôi 5 quyển sách thuộc danh mục phật giáo', type='TextMessage'), ToolCallRequestEvent(source='sql_database_assistant', models_usage=RequestUsage(prompt_tokens=2889, completion_tokens=31), metadata={}, content=[FunctionCall(id='call_IMc3lVesarSNp81a93YTwBMM', arguments='{"query":"SELECT name FROM books WHERE categories LIKE \'%phật giáo%\' LIMIT 5;"}', name='execute_sql_query')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_database_assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="[('Phật Thuyết Đại Thừa Vô Lượng Thọ Trang Nghiêm Thanh Tịnh Bình Đẳng Giác Kinh (Quyển 1)',), ('Lược Sử Tôn Giáo',), ('Sách Muôn Kiếp Nhân Sinh (Bìa Mềm) - Nguyên Phong',), ('Thiền Sư Và Em Bé 5 Tuổi',), ('Hành Trình Về Phương Đông (Tái Bản 2021)',)]", name='execute_sql_query', call_id='call_IMc3lVesarSNp81a93YTwBMM', is_error=False)], type='ToolC

In [ ]:
task = "tôi vừa hỏi câu gì?"
await Console(team.run_stream(task=task))

---------- user ----------
tôi vừa hỏi câu gì?
---------- SummarizeAgent ----------
Bạn đã hỏi về việc giới thiệu 5 quyển sách thuộc danh mục Phật giáo. Nếu bạn cần thêm thông tin hoặc có câu hỏi khác, hãy cho tôi biết! 

TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='tôi vừa hỏi câu gì?', type='TextMessage'), TextMessage(source='SummarizeAgent', models_usage=RequestUsage(prompt_tokens=1649, completion_tokens=41), metadata={}, content='Bạn đã hỏi về việc giới thiệu 5 quyển sách thuộc danh mục Phật giáo. Nếu bạn cần thêm thông tin hoặc có câu hỏi khác, hãy cho tôi biết! \n\nTERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")

In [ ]:
task = "ăndgjángdfngdfngjdf"
await Console(team.run_stream(task=task))

---------- user ----------
ăndgjángdfngdfngjdf
---------- SummarizeAgent ----------
Xin lỗi, có vẻ như bạn đã nhập một chuỗi ký tự không rõ ràng. Nếu bạn có câu hỏi hoặc yêu cầu nào cụ thể, hãy cho tôi biết để tôi có thể hỗ trợ bạn tốt hơn! 

TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='ăndgjángdfngdfngjdf', type='TextMessage'), TextMessage(source='SummarizeAgent', models_usage=RequestUsage(prompt_tokens=1713, completion_tokens=50), metadata={}, content='Xin lỗi, có vẻ như bạn đã nhập một chuỗi ký tự không rõ ràng. Nếu bạn có câu hỏi hoặc yêu cầu nào cụ thể, hãy cho tôi biết để tôi có thể hỗ trợ bạn tốt hơn! \n\nTERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")